<a href="https://colab.research.google.com/github/yingzibu/MOL2ADMET/blob/main/examples/experiments/results/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
print(torch.__version__)
print('cuda: ', torch.cuda.is_available())

2.1.0+cu118
cuda:  False


In [ ]:
! pip install rdkit --quiet
! pip install PyTDC --quiet
! pip install mycolorpy --quiet
! pip install selfies  --quiet
! pip install pubchempy --quiet
! pip install dgllife --quiet
! pip install molvs --quiet
! pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html --quiet
! pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet

! pip install DeepPurpose --quiet
! pip install git+https://github.com/bp-kelley/descriptastorus --quiet
! pip install pandas-flavor --quiet

## Code

In [4]:
from os import walk
import os
file_types = ['bin', 'pth']

# clean certain type of file in path
def clean_files(path = '/content/drive/MyDrive/ADMET/',
                file_types = ['pth', 'bin', 'pt', 'yml']):
    files = next(walk(path), (None, None, []))[2]
    for file in files:
        if isinstance(file, str):
            file_type = file.split('.')[-1]
            if file_type in file_types:
                os.remove(file); print(f'removed from {path}: {file} ')

In [5]:
cd /content/drive/MyDrive/ADMET

/content/drive/MyDrive/ADMET


In [6]:

from scripts.func_utils import *
from scripts.yaml_utils import *
from scripts.eval_utils import *
from scripts.preprocess_mols import *
from scripts.model_architecture import *
from scripts.dataset import *
from scripts.train import *

from tdc.utils import retrieve_label_name_list
import pandas as pd
from scripts.get_vocab import *

from tdc.single_pred import ADME
from tdc.single_pred import Tox
label_list = retrieve_label_name_list('herg_central')


Vocab type for RNN: smiles


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


## Eval single task

In [7]:
for name in names_reg:
    print('=' *30 , name, '=' *30)
    folder_name = f'{name}_scale' # no uncertainty weight on M5
    repeat_time = 3
    for model_type in model_types:
        print('\n', '.'*30, model_type, '.'*30)
        perfs = []
        for i in range(repeat_time):
            file_name = f'{folder_name}/{model_type}_ST_{i}.yml'
            try:
                p = yml_report(file_name)
            # print(p)
                names = [n for n in p.keys()]
                names = names[:-1] # the last one is 'loss'
                assert set([name]) == set(names)
                perfs.append(p)
            except:
                print('cannot open', file_name)
        if len(perfs) != 0:
            print(model_type, 'run repeat time', repeat_time)
            best_idx = eval_perf_list(perfs, names)
            # print('best model in ', model_type)
            # eval_perf_list(perfs[best_idx], names, {})

============================== Caco2_Wang ==============================

 .............................. MLP ..............................
MLP run repeat time 3
repeated num # 3 idx 2 has the lowest loss from [0.014077597922021215, 0.013949783621253549, 0.013443326557075584]
******************** Caco2_Wang ******************** 
	|       mae      |       rmse      |       r2      
	&0.393$\pm$0.007  &0.500$\pm$0.005  &0.607$\pm$0.008  
 idx 2: &0.385            &0.493            &0.618            


 .............................. AttentiveFP ..............................
AttentiveFP run repeat time 3
repeated num # 3 idx 1 has the lowest loss from [0.010073060190284646, 0.009021656198815986, 0.00943417143035721]
******************** Caco2_Wang ******************** 
	|       mae      |       rmse      |       r2      
	&0.338$\pm$0.006  &0.414$\pm$0.009  &0.730$\pm$0.012  
 idx 1: &0.331            &0.404            &0.743            


 .............................. GIN ...........

In [8]:
for name in names_cls:
    print('=' *30 , name, '=' *30)
    folder_name = f'cls/{name}' # no uncertainty weight on M5
    repeat_time = 3
    for model_type in model_types:
        print('\n', '.'*30, model_type, '.'*30)
        perfs = []
        for i in range(repeat_time):
            file_name = f'{folder_name}/{model_type}_ST_{i}.yml'
            try:
                p = yml_report(file_name)
                perfs.append(p)
            except:
                print('cannot open', file_name)

        if len(perfs) != 0:
            best_idx = eval_perf_list(perfs, name)
            eval_perf_list(perfs[best_idx], name, {})
            print('.'*30, model_type, '.'*30)
            print('\n')

============================== CYP2C19_Veith ==============================

 .............................. MLP ..............................
repeated num # 3 idx 2 has the lowest loss from [9.649714589118958, 9.630590319633484, 9.627918720245361]
******************** CYP2C19_Veith ******************** 
	|       acc      |       auc      |       ap      
	&0.782$\pm$0.002  &0.856$\pm$0.001  &0.811$\pm$0.001  
 idx 2: &0.781            &0.855            &0.810            

******************** CYP2C19_Veith ******************** 
	|  acc  |  w_acc  |  prec  |  recall  |  sp  |  f1  |  auc  |  mcc  |  ap  
single: &0.781   &0.783   &0.742   &0.802   &0.764   &0.771   &0.855   &0.564   &0.810   
.............................. MLP ..............................



 .............................. AttentiveFP ..............................
repeated num # 3 idx 2 has the lowest loss from [8.564451843500137, 8.541282385587692, 8.51417589187622]
******************** CYP2C19_Veith *************

## Eval metabolism 5 MT: multi task, means model could predict 5 tasks simultaneously

In [9]:
folder_name = '/content/drive/MyDrive/ADMET/M5/uw_no' # no uncertainty weight on M5
repeat_time = 3
for model_type in model_types:
    print('\n', '.'*30, model_type, '.'*30)
    perfs = []
    for i in range(repeat_time):
        file_name = f'{folder_name}/{model_type}_MT_{i}.yml'
        try:
            p = yml_report(file_name)
            names = [n for n in p.keys()]
            names = names[:-1] # the last one is 'loss'
            perfs.append(p)
        except:
            print('cannot open', file_name)
    if len(perfs) != 0:
        print(model_type, 'run repeat time', repeat_time)
        best_idx = eval_perf_list(perfs, names)
        print('best model in ', model_type)
        eval_perf_list(perfs[best_idx], names, {})
    print('\n', '.'*30, model_type, '.'*30, '\n')


 .............................. MLP ..............................
MLP run repeat time 3
repeated num # 3 idx 2 has the lowest loss from [0.28572497976152866, 0.28331841385527834, 0.2782382261875558]
******************** CYP1A2_Veith ******************** 
	|       acc      |       auc      |       ap      
	&0.825$\pm$0.006  &0.910$\pm$0.002  &0.898$\pm$0.003  
 idx 2: &0.821            &0.913            &0.903            

******************** CYP2C19_Veith ******************** 
	|       acc      |       auc      |       ap      
	&0.801$\pm$0.004  &0.871$\pm$0.006  &0.839$\pm$0.012  
 idx 2: &0.807            &0.879            &0.856            

******************** CYP2C9_Veith ******************** 
	|       acc      |       auc      |       ap      
	&0.806$\pm$0.007  &0.875$\pm$0.006  &0.753$\pm$0.014  
 idx 2: &0.816            &0.882            &0.769            

******************** CYP2D6_Veith ******************** 
	|       acc      |       auc      |       ap      
	&0.85

In [10]:
folder_name = '/content/drive/MyDrive/ADMET/M5/uw' # uncertainty weight on M5
repeat_time = 3
names = ['CYP2C19_Veith',
 'CYP2D6_Veith',
 'CYP3A4_Veith',
 'CYP1A2_Veith',
 'CYP2C9_Veith']
for model_type in model_types:
    print('\n', '.'*30, model_type, '.'*30)
    perfs = []
    for i in range(repeat_time):
        file_name = f'{folder_name}/{model_type}_MT_{i}.yml'
        try:
            p = yml_report(file_name)
            # names = [n for n in p.keys()]
            # names = names[:-1] # the last one is 'loss'
            perfs.append(p)
        except:
            print('cannot open', file_name)
    if len(perfs) != 0:
        print(model_type, 'run repeat time', repeat_time)
        best_idx = eval_perf_list(perfs, names)
        print('best model in ', model_type)
        eval_perf_list(perfs[best_idx], names, {})
    print('\n', '.'*30, model_type, '.'*30, '\n')


 .............................. MLP ..............................
MLP run repeat time 3
repeated num # 3 idx 1 has the lowest loss from [0.2771954964080902, 0.27616064526038625, 0.27656631426860206]
******************** CYP2C19_Veith ******************** 
	|       acc      |       auc      |       ap      
	&0.802$\pm$0.005  &0.876$\pm$0.005  &0.844$\pm$0.007  
 idx 1: &0.807            &0.877            &0.846            

******************** CYP2D6_Veith ******************** 
	|       acc      |       auc      |       ap      
	&0.860$\pm$0.002  &0.836$\pm$0.001  &0.630$\pm$0.003  
 idx 1: &0.859            &0.835            &0.634            

******************** CYP3A4_Veith ******************** 
	|       acc      |       auc      |       ap      
	&0.773$\pm$0.004  &0.847$\pm$0.003  &0.788$\pm$0.003  
 idx 1: &0.771            &0.848            &0.791            

******************** CYP1A2_Veith ******************** 
	|       acc      |       auc      |       ap      
	&0.82

## Evaluate 3 toxicity

In [11]:

# folder_name = 'T3/uw'
# names = ['LD50_Zhu','hERG_Karim', 'AMES']

# for model_type in model_types:
#     ps = train_mt(names, model_type, folder_name, repeat_time=3, retrain=False, uw=True)


In [12]:
folder_name = '/content/drive/MyDrive/ADMET/T3/uw_no' # uncertainty weight on tox mix tasks
repeat_time = 3
names = ['LD50_Zhu','hERG_Karim', 'AMES']
for model_type in model_types:
    print('\n', '.'*30, model_type, '.'*30, '\n')
    perfs = []
    for i in range(repeat_time):
        file_name = f'{folder_name}/{model_type}_MT_{i}.yml'
        try:
            p = yml_report(file_name)
            names_in = [n for n in p.keys()][:-1] # the last one is 'loss'
            try: assert sorted(names_in) == sorted(names)
            except:
                print('name discripancy!:', names_in, 'not the same as ', names)
                names = names_in
            perfs.append(p)
        except:
            print('cannot open', file_name)
    if len(perfs) != 0:
        print(model_type, 'run repeat time', repeat_time)
        best_idx = eval_perf_list(perfs, names)
        print('best model in ', model_type)
        eval_perf_list(perfs[best_idx], names, {})
    print('\n', '.'*30, model_type, '.'*30, '\n')


 .............................. MLP .............................. 

MLP run repeat time 3
repeated num # 3 idx 1 has the lowest loss from [0.5739445134320043, 0.5448673116889867, 0.5673596611754461]
******************** LD50_Zhu ******************** 
	|       mae      |       rmse      |       r2      
	&0.625$\pm$0.025  &0.815$\pm$0.032  &0.255$\pm$0.059  
 idx 1: &0.615            &0.806            &0.272            

******************** hERG_Karim ******************** 
	|       acc      |       auc      |       ap      
	&0.760$\pm$0.004  &0.836$\pm$0.004  &0.838$\pm$0.004  
 idx 1: &0.765            &0.840            &0.842            

******************** AMES ******************** 
	|       acc      |       auc      |       ap      
	&0.814$\pm$0.004  &0.873$\pm$0.004  &0.882$\pm$0.002  
 idx 1: &0.817            &0.875            &0.885            

best model in  MLP
******************** LD50_Zhu ******************** 
	|  mae  |  mse  |  rmse  |  r2  
single: &0.615   &0.650

In [13]:
folder_name = '/content/drive/MyDrive/ADMET/T3/uw' # uncertainty weight on tox mix tasks
repeat_time = 3
names = ['LD50_Zhu','hERG_Karim', 'AMES']
for model_type in model_types:
    print('\n', '.'*30, model_type, '.'*30, '\n')
    perfs = []
    for i in range(repeat_time):
        file_name = f'{folder_name}/{model_type}_MT_{i}.yml'
        try:
            p = yml_report(file_name)
            names_in = [n for n in p.keys()][:-1] # the last one is 'loss'
            try: assert sorted(names_in) == sorted(names)
            except:
                print('name discripancy!:', names_in, 'not the same as ', names)
                names = names_in
            perfs.append(p)
        except:
            print('cannot open', file_name)
    if len(perfs) != 0:
        print(model_type, 'run repeat time', repeat_time)
        best_idx = eval_perf_list(perfs, names)
        print('best model in ', model_type)
        eval_perf_list(perfs[best_idx], names, {})
    print('\n', '.'*30, model_type, '.'*30, '\n')


 .............................. MLP .............................. 

MLP run repeat time 3
repeated num # 3 idx 1 has the lowest loss from [0.0025294304253872146, 0.002378456044508229, 0.0030601767463270912]
******************** LD50_Zhu ******************** 
	|       mae      |       rmse      |       r2      
	&0.493$\pm$0.003  &0.672$\pm$0.013  &0.495$\pm$0.020  
 idx 1: &0.491            &0.661            &0.512            

******************** hERG_Karim ******************** 
	|       acc      |       auc      |       ap      
	&0.733$\pm$0.020  &0.808$\pm$0.018  &0.810$\pm$0.018  
 idx 1: &0.719            &0.788            &0.790            

******************** AMES ******************** 
	|       acc      |       auc      |       ap      
	&0.779$\pm$0.028  &0.843$\pm$0.023  &0.865$\pm$0.014  
 idx 1: &0.745            &0.813            &0.848            

best model in  MLP
******************** LD50_Zhu ******************** 
	|  mae  |  mse  |  rmse  |  r2  
single: &0.491 

# AD
names = ['BBB_Martins',
 'Bioavailability_Ma',
 'Pgp_Broccatelli',
 'HIA_Hou',
 'PAMPA_NCATS']

In [14]:
names = ['BBB_Martins',
 'Bioavailability_Ma',
 'Pgp_Broccatelli',
 'HIA_Hou',
 'PAMPA_NCATS']

In [15]:
metrics_one = d.copy()
metrics_one['reg'] = [3]
metrics_one['cls'] = [6]
folder_name = '/content/drive/MyDrive/ADMET/AD/uw_no' # uncertainty weight on tox mix tasks
repeat_time = 3
names = ['BBB_Martins',
 'Bioavailability_Ma',
 'Pgp_Broccatelli',
 'HIA_Hou',
 'PAMPA_NCATS']
for model_type in model_types:
    print('\n', '.'*30, model_type, '.'*30, '\n')
    perfs = []
    for i in range(repeat_time):
        file_name = f'{folder_name}/{model_type}_MT_{i}.yml'
        try:
            p = yml_report(file_name)
            names_in = [n for n in p.keys()][:-1] # the last one is 'loss'
            try: assert sorted(names_in) == sorted(names)
            except:
                print('name discripancy!:', names_in, 'not the same as ', names)
                names = names_in
            perfs.append(p)
        except:
            print('cannot open', file_name)
    if len(perfs) != 0:
        print(model_type, 'run repeat time', repeat_time)
        best_idx = eval_perf_list(perfs, names, metrics_one)
        print('best model in ', model_type)
        eval_perf_list(perfs[best_idx], names, {})
    print('\n', '.'*30, model_type, '.'*30, '\n')


 .............................. MLP .............................. 

MLP run repeat time 3
repeated num # 3 idx 0 has the lowest loss from [0.9777523070573807, 0.9838002979755401, 1.0087730139493942]
******************** BBB_Martins ******************** 
	|       auc      
	&0.848$\pm$0.002  
 idx 0: &0.845            

******************** Bioavailability_Ma ******************** 
	|       auc      
	&0.592$\pm$0.030  
 idx 0: &0.627            

******************** Pgp_Broccatelli ******************** 
	|       auc      
	&0.839$\pm$0.035  
 idx 0: &0.875            

******************** HIA_Hou ******************** 
	|       auc      
	&0.975$\pm$0.002  
 idx 0: &0.973            

******************** PAMPA_NCATS ******************** 
	|       auc      
	&0.730$\pm$0.002  
 idx 0: &0.731            

best model in  MLP
******************** BBB_Martins ******************** 
	|  acc  |  w_acc  |  prec  |  recall  |  sp  |  f1  |  auc  |  mcc  |  ap  
single: &0.837   &0.718   &0.84

In [16]:
metrics_one = d.copy()
metrics_one['reg'] = [3]
metrics_one['cls'] = [6]

In [17]:
metrics_one = d.copy()
metrics_one['reg'] = [3]
metrics_one['cls'] = [6]

folder_name = '/content/drive/MyDrive/ADMET/AD/uw'
repeat_time = 3
names = ['BBB_Martins',
 'Bioavailability_Ma',
 'Pgp_Broccatelli',
 'HIA_Hou',
 'PAMPA_NCATS']
for model_type in model_types:
    print('\n', '.'*30, model_type, '.'*30, '\n')
    perfs = []
    for i in range(repeat_time):
        file_name = f'{folder_name}/{model_type}_MT_{i}.yml'
        try:
            p = yml_report(file_name)
            names_in = [n for n in p.keys()][:-1] # the last one is 'loss'
            try: assert sorted(names_in) == sorted(names)
            except:
                print('name discripancy!:', names_in, 'not the same as ', names)
                names = names_in
            perfs.append(p)
        except:
            print('cannot open', file_name)
    if len(perfs) != 0:
        print(model_type, 'run repeat time', repeat_time)
        best_idx = eval_perf_list(perfs, names, metrics_one)
        print('best model in ', model_type)
        eval_perf_list(perfs[best_idx], names, {})
    print('\n', '.'*30, model_type, '.'*30, '\n')


 .............................. MLP .............................. 

MLP run repeat time 3
repeated num # 3 idx 2 has the lowest loss from [0.07469304744154215, 0.07753822468221187, 0.0645070462487638]
******************** BBB_Martins ******************** 
	|       auc      
	&0.856$\pm$0.005  
 idx 2: &0.852            

******************** Bioavailability_Ma ******************** 
	|       auc      
	&0.617$\pm$0.024  
 idx 2: &0.647            

******************** Pgp_Broccatelli ******************** 
	|       auc      
	&0.839$\pm$0.006  
 idx 2: &0.846            

******************** HIA_Hou ******************** 
	|       auc      
	&0.979$\pm$0.003  
 idx 2: &0.979            

******************** PAMPA_NCATS ******************** 
	|       auc      
	&0.731$\pm$0.028  
 idx 2: &0.742            

best model in  MLP
******************** BBB_Martins ******************** 
	|  acc  |  w_acc  |  prec  |  recall  |  sp  |  f1  |  auc  |  mcc  |  ap  
single: &0.840   &0.729   &0.

AD_mix: 2 classification and 2 regression tasks

    names = ['BBB_Martins', 'PAMPA_NCATS', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB']

In [18]:
names = ['BBB_Martins', 'PAMPA_NCATS', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB']

metrics_one = d.copy()
metrics_one['reg'] = [3]
metrics_one['cls'] = [6]

folder_name = '/content/drive/MyDrive/ADMET/AD_mix/uw_no'
repeat_time = 3
names = ['BBB_Martins', 'PAMPA_NCATS', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB']
for model_type in model_types:
    print('\n', '.'*30, model_type, '.'*30, '\n')
    perfs = []
    for i in range(repeat_time):
        file_name = f'{folder_name}/{model_type}_MT_{i}.yml'
        try:
            p = yml_report(file_name)
            names_in = [n for n in p.keys()][:-1] # the last one is 'loss'
            try: assert sorted(names_in) == sorted(names)
            except:
                print('name discripancy!:', names_in, 'not the same as ', names)
                names = names_in
            perfs.append(p)
        except:
            print('cannot open', file_name)
    if len(perfs) != 0:
        print(model_type, 'run repeat time', repeat_time)
        best_idx = eval_perf_list(perfs, names, metrics_one)
        print('best model in ', model_type)
        eval_perf_list(perfs[best_idx], names, {})
    print('\n', '.'*30, model_type, '.'*30, '\n')

folder_name = '/content/drive/MyDrive/ADMET/AD_mix/uw'
repeat_time = 3
names = ['BBB_Martins', 'PAMPA_NCATS', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB']
for model_type in model_types:
    print('\n', '.'*30, model_type, '.'*30, '\n')
    perfs = []
    for i in range(repeat_time):
        file_name = f'{folder_name}/{model_type}_MT_{i}.yml'
        try:
            p = yml_report(file_name)
            names_in = [n for n in p.keys()][:-1] # the last one is 'loss'
            try: assert sorted(names_in) == sorted(names)
            except:
                print('name discripancy!:', names_in, 'not the same as ', names)
                names = names_in
            perfs.append(p)
        except:
            print('cannot open', file_name)
    if len(perfs) != 0:
        print(model_type, 'run repeat time', repeat_time)
        best_idx = eval_perf_list(perfs, names, metrics_one)
        print('best model in ', model_type)
        eval_perf_list(perfs[best_idx], names, {})
    print('\n', '.'*30, model_type, '.'*30, '\n')


 .............................. MLP .............................. 

MLP run repeat time 3
repeated num # 3 idx 0 has the lowest loss from [0.030570224963192796, 0.03120217865718336, 0.03195274227844744]
******************** BBB_Martins ******************** 
	|       auc      
	&0.870$\pm$0.002  
 idx 0: &0.868            

******************** PAMPA_NCATS ******************** 
	|       auc      
	&0.757$\pm$0.016  
 idx 0: &0.770            

******************** Lipophilicity_AstraZeneca ******************** 
	|       r2      
	&0.252$\pm$0.044  
 idx 0: &0.190            

******************** Solubility_AqSolDB ******************** 
	|       r2      
	&0.377$\pm$0.046  
 idx 0: &0.350            

best model in  MLP
******************** BBB_Martins ******************** 
	|  acc  |  w_acc  |  prec  |  recall  |  sp  |  f1  |  auc  |  mcc  |  ap  
single: &0.837   &0.740   &0.858   &0.937   &0.544   &0.896   &0.868   &0.539   &0.943   
******************** PAMPA_NCATS **************